# Deep Neural Networks for Wage Prediction

So far we considered many machine learning method, e.g Lasso and Random Forests, to build a predictive model. In this lab, we extend our toolbox by predicting wages by a neural network.

## Data preparation

Again, we consider data from the U.S. March Supplement of the Current Population Survey (CPS) in 2015.



This notebook contains an example for teaching.


**Simple Case Study using Wage Data from 2015 - proceeding**

So far we considered many machine learning method, e.g Lasso and Random Forests, to build a predictive model. In this lab, we extend our toolbox by predicting wages by a neural network.

**Data preparation**

Again, we consider data from the U.S. March Supplement of the Current Population Survey (CPS) in 2015.

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')

In [2]:
rdata_read = pyreadr.read_r("../data/wage2015_subsample_inference.Rdata")
data = rdata_read[ 'data' ]
n = data.shape[0]

In [3]:
# Import relevant packages for splitting data
import random
import math
# Set Seed
# to make the results replicable (generating random numbers)
np.random.seed(0)
random = np.random.randint(0, data.shape[0], size=math.floor(data.shape[0]))
data["random"] = random
random    # the array does not change 

array([2732, 2607, 1653, ..., 4184, 2349, 3462])

In [4]:
data_2 = data.sort_values(by=['random'])
data_2

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,...,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2,random
rownames,,,,,,,,,,,,,,,,,,,,,
2223,26.442308,3.274965,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,29.0,8.4100,24.389000,70.728100,340,1,8660,20,0
3467,19.230769,2.956512,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,33.5,11.2225,37.595375,125.944506,9620,22,1870,5,0
13501,48.076923,3.872802,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,2.0,0.0400,0.008000,0.001600,3060,10,8190,18,0
15588,12.019231,2.486508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,29.0,8.4100,24.389000,70.728100,6440,19,770,4,2
16049,39.903846,3.686473,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,12.0,1.4400,1.728000,2.073600,1820,5,7860,17,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27533,21.634615,3.074295,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,10.0,1.0000,1.000000,1.000000,1860,5,7870,17,5146
7218,13.461538,2.599837,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,28.0,7.8400,21.952000,61.465600,4220,14,1170,5,5146
7204,27.403846,3.310683,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,4.0,0.1600,0.064000,0.025600,1760,5,7460,14,5146


In [5]:
# Create training and testing sample 
data_train = data_2[ : math.floor(n*3/4)]    # training sample
data_test =  data_2[ math.floor(n*3/4) : ]   # testing sample
print(data_train.shape)
print(data_test.shape)

(3862, 21)
(1288, 21)


In [6]:
data_train = data_train.iloc[:, 0:16]
data_test = data_test.iloc[:, 0:16] 
data_test

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
rownames,,,,,,,,,,,,,,,,
9210,29.807692,3.394766,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,4.84,10.648,23.4256
16484,43.269231,3.767442,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,17.0,2.89,4.913,8.3521
16448,24.038462,3.179655,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,11.0,1.21,1.331,1.4641
27392,10.097115,2.312250,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,30.0,9.00,27.000,81.0000
11596,8.653846,2.158004,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.36,0.216,0.1296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27533,21.634615,3.074295,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.0,1.00,1.000,1.0000
7218,13.461538,2.599837,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28.0,7.84,21.952,61.4656
7204,27.403846,3.310683,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.16,0.064,0.0256


In [7]:
# normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler =  MinMaxScaler().fit(data_train)
scaler =  MinMaxScaler().fit(data_test)

# scaler = preprocessing.StandardScaler().fit(data_train)
# scaler = preprocessing.StandardScaler().fit(data_test)

data_train_scaled = scaler.transform(data_train)
data_test_scaled = scaler.transform(data_test)


In [8]:
columns = list(data_train)

In [9]:
data_train_scaled = pd.DataFrame(data_train_scaled, columns = columns)
data_test_scaled = pd.DataFrame(data_test_scaled, columns = columns)
data_test_scaled

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
0,0.054875,0.447819,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.500000,0.250000,0.125000,0.062500
1,0.082510,0.521313,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.386364,0.149277,0.057675,0.022284
2,0.043032,0.405398,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.250000,0.062500,0.015625,0.003906
3,0.014412,0.234342,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.681818,0.464876,0.316961,0.216110
4,0.011449,0.203924,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.136364,0.018595,0.002536,0.000346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,0.038097,0.384621,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.227273,0.051653,0.011739,0.002668
1284,0.021318,0.291056,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.636364,0.404959,0.257701,0.163992
1285,0.049940,0.431238,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.090909,0.008264,0.000751,0.000068
1286,0.027958,0.333518,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.636364,0.404959,0.257701,0.163992


Then, we construct the inputs for our network.

In [10]:
formula_basic = "lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we"
Y_train, model_X_basic_train = patsy.dmatrices(formula_basic, data_train_scaled, return_type='dataframe')
Y_test, model_X_basic_test = patsy.dmatrices(formula_basic, data_test_scaled, return_type='dataframe')

## Neural Networks

First, we need to determine the structure of our network. We are using the R/python package *keras* to build a simple sequential neural network with three dense layers.

In [11]:
model_X_basic_train.shape[1]

10

In [12]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim = model_X_basic_train.shape[1], activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
# model.add(Dense(5, activation = 'relu'))

model.add(Dense(1))

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [14]:
# compile the keras model
opt = keras.optimizers.Adam(learning_rate=0.005)
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None)

Let us have a look at the structure of our network in detail.

In [15]:
model.compile(loss=mse, optimizer= opt , metrics=mae)
model.summary(line_length=None, positions=None, print_fn=None)

Model: "sequential"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 dense (Dense)               (None, 20)                220       


 dense_1 (Dense)             (None, 10)                210       


 dense_2 (Dense)             (None, 1)                 11        


Total params: 441


Trainable params: 441


Non-trainable params: 0


_________________________________________________________________


It is worth to notice that we have in total $441$ trainable parameters.

Now, let us train the network. Note that this takes some computation time. Thus, we are using gpu to speed up. The exact speed-up varies based on a number of factors including model architecture, batch-size, input pipeline complexity, etc.

In [16]:
# fit the keras model on the dataset
num_epochs = 1000

Check this [link](https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network), to understand batch_size argument

In [17]:
# fit the keras model on the dataset
model.fit(model_X_basic_train, Y_train, epochs=150, batch_size=10)

Epoch 1/150


  1/387 [..............................] - ETA: 5:23 - loss: 0.0642 - mean_absolute_error: 0.2107

 37/387 [=>............................] - ETA: 0s - loss: 0.0321 - mean_absolute_error: 0.1412  

 72/387 [====>.........................] - ETA: 0s - loss: 0.0237 - mean_absolute_error: 0.1188

108/387 [=======>......................] - ETA: 0s - loss: 0.0202 - mean_absolute_error: 0.1089

...

303/387 [======================>.......] - ETA: 0s - loss: 0.0101 - mean_absolute_error: 0.0766

330/387 [========================>.....] - ETA: 0s - loss: 0.0101 - mean_absolute_error: 0.0765

355/387 [==========================>...] - ETA: 0s - loss: 0.0100 - mean_absolute_error: 0.0765

386/387 [============================>.] - ETA: 0s - loss: 0.0102 - mean_absolute_error: 0.0771

387/387 [==============================] - 1s 2ms/step - loss: 0.0102 - mean_absolute_error: 0.0771


In [18]:
model.metrics_names

['loss', 'mean_absolute_error']

In [19]:
model.evaluate(model_X_basic_test, Y_test, verbose = 0)

[0.010300195775926113, 0.07794734090566635]

In [20]:
pred_nn = model.predict(model_X_basic_test)
pred_nn

 1/41 [..............................] - ETA: 7s

38/41 [==========================>...] - ETA: 0s

41/41 [==============================] - 0s 2ms/step


array([[0.29051456],
       [0.39695188],
       [0.39343563],
       ...,
       [0.39221323],
       [0.41987908],
       [0.38774517]], dtype=float32)

In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [22]:
resid_basic = (Y_test-pred_nn)**2

In [23]:
MSE_nn_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[0] ) ).fit().summary2().tables[1].iloc[0, 0:2]
MSE_nn_basic

Coef.       0.010300
Std.Err.    0.000561
Name: const, dtype: float64

In [24]:
R2_nn_basic = 1 - ( MSE_nn_basic[0]/Y_test.var() )
print( f"The R^2 using NN is equal to = {R2_nn_basic[0]}" ) # MSE NN (basic model) 

The R^2 using NN is equal to = 0.14397281074722568
